In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras_tuner import RandomSearch

train_dir = r'E:\AI\dataset_skeleton_sep\face\LyingLegRaise\training'
val_dir = r'E:\AI\dataset_skeleton_sep\face\LyingLegRaise\validation'
test_dir = r'E:\AI\dataset_skeleton_sep\face\LyingLegRaise\test'

# ImageDataGenerator 초기화
datagen = ImageDataGenerator(rescale=1./255)  # 이미지를 0과 1 사이의 값으로 정규화

# 훈련, 검증, 테스트 데이터셋을 위한 제너레이터 생성
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=True)

validation_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=False)

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=False)


Found 15894 images belonging to 16 classes.
Found 3415 images belonging to 16 classes.
Found 3406 images belonging to 16 classes.


In [2]:
def build_model(hp):
    base_model = tf.keras.applications.InceptionV3(weights="imagenet",
                                                    include_top=False, 
                                                    input_shape=(128, 128, 3))
    for layer in base_model.layers:
        base_model.trainable = False
    for layer in base_model.layers[-9:]:
        base_model.trainable = True

    model = models.Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    # Dense 레이어의 유닛 수를 하이퍼파라미터로 사용
    model.add(layers.Dense(units=hp.Int('units', min_value=128, max_value=512, step=128), activation='relu'))
    model.add(layers.Dense(16, activation='softmax'))
    
    # learning_rate를 하이퍼파라미터로 사용
    model.compile(optimizer=optimizers.Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-3, sampling='LOG')),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [3]:
import keras_tuner as kt

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # 시도할 하이퍼파라미터 조합의 최대 개수
    executions_per_trial=1,  # 각 하이퍼파라미터 설정을 평가하기 위해 모델을 훈련시킬 횟수
    directory='random_search',  # 튜닝 세션의 결과를 저장할 디렉토리 이름
    project_name='GN_LLR_RS_01', # 프로젝트 이름   
)
tuner.search_space_summary()

Reloading Tuner from random_search\GN_LLR_RS_01\tuner0.json
Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 512, 'step': 128, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.001, 'step': None, 'sampling': 'log'}


In [4]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

tuner.search(train_generator,
             steps_per_epoch=train_generator.samples // train_generator.batch_size,
             validation_data=validation_generator, 
             validation_steps=validation_generator.samples // validation_generator.batch_size,
             epochs=25,
             callbacks=[early_stopping])


# 최적의 하이퍼파라미터 가져오기
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# 최적의 하이퍼파라미터로 모델 빌드
model = tuner.hypermodel.build(best_hps)

In [5]:
# 모든 최적 하이퍼파라미터 출력
for hp in best_hps.values:
    print(f"The optimal value for {hp} is {best_hps.get(hp)}")
    
# 모델 훈련
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // train_generator.batch_size,
                    validation_data=validation_generator, 
                    validation_steps=validation_generator.samples // validation_generator.batch_size,
                    epochs=25,
                    callbacks=[early_stopping])

# 모델 평가 (테스트 데이터셋)
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print('\n테스트 정확도:', test_acc)

The optimal value for units is 256
The optimal value for learning_rate is 0.00036403875971769393
Epoch 1/25
496/496 [==============================] - 73s 132ms/step - loss: 2.1702 - accuracy: 0.2328 - val_loss: 2.2102 - val_accuracy: 0.2594
Epoch 2/25
496/496 [==============================] - 39s 79ms/step - loss: 1.5774 - accuracy: 0.4233 - val_loss: 3.7259 - val_accuracy: 0.1860
Epoch 3/25
496/496 [==============================] - 37s 75ms/step - loss: 1.2167 - accuracy: 0.5458 - val_loss: 1.2671 - val_accuracy: 0.5507
Epoch 4/25
496/496 [==============================] - 40s 80ms/step - loss: 0.9648 - accuracy: 0.6427 - val_loss: 1.9460 - val_accuracy: 0.4098
Epoch 5/25
496/496 [==============================] - 39s 80ms/step - loss: 0.7891 - accuracy: 0.7122 - val_loss: 1.1606 - val_accuracy: 0.6111
Epoch 6/25
496/496 [==============================] - 38s 77ms/step - loss: 0.6531 - accuracy: 0.7607 - val_loss: 0.9049 - val_accuracy: 0.6899
Epoch 7/25
496/496 [==================